# Composicion musical

NOTA: Por conveniencia mia no voy a traducir los conceptos musicales al espaniol. No sabia nada de musica (aparte de escucharla) antes de investigar el tema por lo que hubiera tomado mas esfuerzo traducir los conceptos.

Para que un modelo pueda componer musica que sea placentera para nuestros oidos, debe pode superar varios de los problemas tecnicos que vimos cuando trabajamos con texto. En particula el modelo debe ser capaz de aprender de y recrear la estructura secuencial de la musica y tambien debe ser capaz de escoger de un conjunto discreto de posibilidades para notas subsecuentes.

Sin embargo, la generacion muscial presenta desafios adicionales que no son requeridos para la generacion de texto, principalmente pitch y rythm. La musica es comunmente polifonica, es decir que tiene varias corrientes de notas sonando simultaneamente en diferentes instrumentos, los cuales se combinan para crear armonias que son disonantes (chocantes) o consonantes (armoniosas). La generacion de texto solo nos requiere que manejemos una unica corriente de texto, en vez de las corrientes paralelas de acordes que estan presentes en la musica.

## Preliminares

Antes de emepzar vamos a aprender lo basico de teoria musical. En esta seccion vamos a ver lo esencial de la notacion requerida para leer musica y como podemos representarla de forma numerica, para poder transformar musica a inputs que pueden ser alimentados a un modelo. 

El dataset crudo que vamos a usar es un conjunto de archivos MIDI para los [Cello Suites por J.S. Bach](http://www.jsbach.net/midi/midi_solo_cello.html). Para ver y escuchar la musica generada por el modelo, van a necesitar algun softwar que pueda producir notacion musical. [MuseScore](https://musescore.org) es una buena herramienta para este proposito y se puede descargar de forma gratuita.

## Notacion Musical

Vamos a usar la libreria `music21` para cargar los archivos MIDI a Python para procesamiento.

In [15]:
from music21 import converter, environment, chord, note

In [7]:
us = environment.UserSettings()
us['musescoreDirectPNGPath'] = "/snap/musescore/current/usr/bin/mscore"

In [8]:
dataset_name = "cello"
filename = "cs1-2all"
file = f"../data/{dataset_name}/{filename}.mid"

original_score = converter.parse(file).chordify()

Usamos el metodo `chordify` para "aplastar" todas las notas sonando simultaneamente a acordes dentro de una unica parte, en vez de tenerlas separadas entre varias partes. Ya que esta pieza fue creada por un solo instrumento (cello), podemos justificar hacer esto. Para musica que es polifonica lo mejor seria mantener las partes separadas.

In [11]:
original_score.show('text')

{0.0} <music21.instrument.Violoncello 'Violoncello'>
{0.0} <music21.tempo.MetronomeMark Quarter=250.0>
{0.0} <music21.key.Key of G major>
{0.0} <music21.meter.TimeSignature 4/4>
{0.0} <music21.note.Rest rest>
{3.5} <music21.tempo.MetronomeMark Quarter=77.0>
{3.75} <music21.chord.Chord B3>
{4.0} <music21.chord.Chord G2 D3 B3>
{5.0} <music21.chord.Chord B3>
{5.25} <music21.chord.Chord A3>
{5.5} <music21.chord.Chord G3>
{5.75} <music21.chord.Chord F#3>
{6.0} <music21.chord.Chord G3>
{6.25} <music21.chord.Chord D3>
{6.5} <music21.chord.Chord E3>
{6.75} <music21.chord.Chord F#3>
{7.0} <music21.chord.Chord G3>
{7.25} <music21.chord.Chord A3>
{7.5} <music21.chord.Chord B3>
{7.75} <music21.chord.Chord C4>
{8.0} <music21.chord.Chord D4>
{8.25} <music21.chord.Chord B3>
{8.5} <music21.chord.Chord G3>
{8.75} <music21.chord.Chord F#3>
{9.0} <music21.chord.Chord G3>
{9.25} <music21.chord.Chord E3>
{9.5} <music21.chord.Chord D3>
{9.75} <music21.chord.Chord C3>
{10.0} <music21.chord.Chord B2>
{10.25} 

1. El archivo MIDI empieza con metadata sobre la instrumentacion, tempo, y key de la pieza
2. La octava nota (fila) empieza en el beat 4 de la pieza (zero indexed). Tiene una duracion de 1 beat (ya que la siguiente nota empieza en beat 5) y consista de un chord de low G, D y B
3. La nota (fila) 13 empieza en el beat 6 de la siguiente pieza. Tiene una duracion de un cuarto de beat (ya que la siguiente nota empieza en beat 6.25) y consista de una unica nota G. 
4. La nota en la fila 21 empieza un cuarto de beat antes del beat 8 de la pieza. Tiene una duracion de un cuarto de beat y consiste en una unica nota, high C.

El siguiente codigo loopea a traves del score y extrae el pitch y duracion de cada nota (y rest) en la pieza en dos listas. Notas individuales en chords estan separadas por un punto, para que el chord entero puedo ser almacenado en un string. El numero despues del nombre de cada nota indica el _octave_ en el que la nota esta, ya que los nombres de las notas (A - G) se repiten, esto es neesario para poder identificar el pitch de cada note. Por ejemplo, G2 es un octavo debajo de G3.

In [16]:
notes = []
durations = []

for element in original_score.flat:
    
    if isinstance(element, chord.Chord):
        notes.append('.'.join(n.nameWithOctave for n in element.pitches))
        durations.append(element.duration.quarterLength)

    if isinstance(element, note.Note):
        if element.isRest:
            notes.append(str(element.name))
            durations.append(element.duration.quarterLength)
        else:
            notes.append(str(element.nameWithOctave))
            durations.append(element.duration.quarterLength)

In [21]:
print('\nduration', 'pitch')
for n,d in zip(notes, durations):
    print(d, '\t', n)
    break


duration pitch
0.25 	 B3


In [19]:
import pandas as pd

dataset = pd.DataFrame({"duration": durations,
                        "pitch": notes})

dataset.head()

,duration,pitch
0,0.25,B3
1,1,G2.D3.B3
2,0.25,B3
3,0.25,A3
4,0.25,G3


El dataset resultante ya se ve mas parecido a los datsets de texto que hemos usado. Las palabras son los _pitches_ y deberiamos tratar de construir un modelo que prediga el siguiente pitch, dado una secuencia de pitches anteriores. La misma idea se puede aplicar a la lista de durations.

## RNN Para Generar Musica

Para crear el dataset necesario para entrenar el modelo, primero tenemos que darle un valor entero a cada pitch y duration, exactamente como hemos hecho para cada palabra en los datasets de texto. No importa cuales sean estos valores ya que deberian usar un embedding layer para transformar los enteros a vectores.

Luego creamos el training set partiendo la data en chunks de 32 notas, usando como label la siguiente nota en la secuencia (one-hot encoded), para el pitch y duration.

In [31]:
import glob
import os

In [32]:
def get_music_list(data_folder):
    
    if data_folder == 'chorales':
        file_list = ['bwv' + str(x['bwv']) for x in corpus.chorales.ChoraleList().byBWV.values()]
        parser = corpus
    else:
        file_list = glob.glob(os.path.join(data_folder, "*.mid"))
        parser = converter
    
    return file_list, parser


def get_distinct(elements):
    # Get all pitch names
    element_names = sorted(set(elements))
    n_elements = len(element_names)
    return (element_names, n_elements)


def create_lookups(element_names):
    # create dictionary to map notes and durations to integers
    element_to_int = dict((element, number) for number, element in enumerate(element_names))
    int_to_element = dict((number, element) for number, element in enumerate(element_names))

    return (element_to_int, int_to_element)

In [34]:
# data params
intervals = range(1)
seq_len = 32

## Extraer las notas

In [35]:
music_list, parser = get_music_list("../data/cello/")
print(len(music_list), 'files in total')

notes = []
durations = []

for i, file in enumerate(music_list):
    print(i+1, "Parsing %s" % file)
    original_score = parser.parse(file).chordify()


    for interval in intervals:

        score = original_score.transpose(interval)

        notes.extend(['START'] * seq_len)
        durations.extend([0]* seq_len)

        for element in score.flat:

            if isinstance(element, note.Note):
                if element.isRest:
                    notes.append(str(element.name))
                    durations.append(element.duration.quarterLength)
                else:
                    notes.append(str(element.nameWithOctave))
                    durations.append(element.duration.quarterLength)

            if isinstance(element, chord.Chord):
                notes.append('.'.join(n.nameWithOctave for n in element.pitches))
                durations.append(element.duration.quarterLength)

36 files in total
1 Parsing ../data/cello/cs1-3cou.mid
2 Parsing ../data/cello/cs3-3cou.mid
3 Parsing ../data/cello/cs5-6gig.mid
4 Parsing ../data/cello/cs2-2all.mid
5 Parsing ../data/cello/cs5-5gav.mid
6 Parsing ../data/cello/cs6-3cou.mid
7 Parsing ../data/cello/cs1-4sar.mid
8 Parsing ../data/cello/cs1-6gig.mid
9 Parsing ../data/cello/cs4-5bou.mid
10 Parsing ../data/cello/cs3-2all.mid
11 Parsing ../data/cello/cs2-1pre.mid
12 Parsing ../data/cello/cs1-5men.mid
13 Parsing ../data/cello/cs1-1pre.mid
14 Parsing ../data/cello/cs6-5gav.mid
15 Parsing ../data/cello/cs4-1pre.mid
16 Parsing ../data/cello/cs1-2all.mid
17 Parsing ../data/cello/cs6-6gig.mid
18 Parsing ../data/cello/cs6-2all.mid
19 Parsing ../data/cello/cs4-6gig.mid
20 Parsing ../data/cello/cs4-2all.mid
21 Parsing ../data/cello/cs3-1pre.mid
22 Parsing ../data/cello/cs5-4sar.mid
23 Parsing ../data/cello/cs2-4sar.mid
24 Parsing ../data/cello/cs5-3cou.mid
25 Parsing ../data/cello/cs3-4sar.mid
26 Parsing ../data/cello/cs4-3cou.mid
27 

## Crear los lookup tables

In [36]:
# get the distinct sets of notes and durations
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

# make the lookup dictionaries for notes and dictionaries and save
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

In [37]:
print('\nnote_to_int')
note_to_int


note_to_int


{'A2': 0,
 'A2.A3': 1,
 'A2.B2': 2,
 'A2.C3': 3,
 'A2.C3.D3.E3': 4,
 'A2.D3': 5,
 'A2.E-3': 6,
 'A2.E3': 7,
 'A2.E3.A3': 8,
 'A2.E3.C#4': 9,
 'A2.E3.C#4.A4': 10,
 'A2.E3.C#4.E4': 11,
 'A2.E3.C#4.G#4': 12,
 'A2.E3.C4': 13,
 'A2.E3.D4': 14,
 'A2.F#3': 15,
 'A2.F#3.C4': 16,
 'A2.F#3.D4': 17,
 'A2.F#3.D4.A4': 18,
 'A2.F#3.D4.E4': 19,
 'A2.F#3.D4.F#4': 20,
 'A2.F#4': 21,
 'A2.F3': 22,
 'A2.F3.C4': 23,
 'A2.F3.D4': 24,
 'A2.F3.D4.A4': 25,
 'A2.G3': 26,
 'A2.G3.C#4': 27,
 'A2.G3.D4': 28,
 'A3': 29,
 'A3.B-3': 30,
 'A3.B3': 31,
 'A3.B3.A4': 32,
 'A3.B3.C#4': 33,
 'A3.B3.C4': 34,
 'A3.B3.F#4.G4': 35,
 'A3.B3.G4': 36,
 'A3.C#4': 37,
 'A3.C#4.E4': 38,
 'A3.C4': 39,
 'A3.D4': 40,
 'A3.D4.E4': 41,
 'A3.D4.F#4': 42,
 'A3.E4': 43,
 'A3.E4.F#4': 44,
 'A3.E4.F#4.G4': 45,
 'A3.E4.G4': 46,
 'A3.F#4': 47,
 'A3.F#4.G4': 48,
 'A3.F4': 49,
 'A3.G#4.A4': 50,
 'A3.G4': 51,
 'A4': 52,
 'A4.B4': 53,
 'B-2': 54,
 'B-2.A3': 55,
 'B-2.B-3': 56,
 'B-2.D3': 57,
 'B-2.D3.A3': 58,
 'B-2.D3.E-3.G#3': 59,
 'B-2.D3.G#3': 

## Preparar las secuencias para el NN

In [38]:
import numpy as np

In [41]:
def to_categorical(y, num_classes=None, dtype='float32'):
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

In [42]:
def prepare_sequences(notes, durations, lookups, distincts, seq_len =32):
    """ Prepare the sequences used to train the Neural Network """

    note_to_int, int_to_note, duration_to_int, int_to_duration = lookups
    note_names, n_notes, duration_names, n_durations = distincts

    notes_network_input = []
    notes_network_output = []
    durations_network_input = []
    durations_network_output = []

    # create input sequences and the corresponding outputs
    for i in range(len(notes) - seq_len):
        notes_sequence_in = notes[i:i + seq_len]
        notes_sequence_out = notes[i + seq_len]
        notes_network_input.append([note_to_int[char] for char in notes_sequence_in])
        notes_network_output.append(note_to_int[notes_sequence_out])

        durations_sequence_in = durations[i:i + seq_len]
        durations_sequence_out = durations[i + seq_len]
        durations_network_input.append([duration_to_int[char] for char in durations_sequence_in])
        durations_network_output.append(duration_to_int[durations_sequence_out])

    n_patterns = len(notes_network_input)

    # reshape the input into a format compatible with LSTM layers
    notes_network_input = np.reshape(notes_network_input, (n_patterns, seq_len))
    durations_network_input = np.reshape(durations_network_input, (n_patterns, seq_len))
    network_input = [notes_network_input, durations_network_input]

    notes_network_output = to_categorical(notes_network_output, num_classes=n_notes)
    durations_network_output = to_categorical(durations_network_output, num_classes=n_durations)
    network_output = [notes_network_output, durations_network_output]

    return (network_input, network_output)

In [43]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts, seq_len)

In [44]:
print('pitch input')
print(network_input[0][0])
print('duration input')
print(network_input[1][0])
print('pitch output')
print(network_output[0][0])
print('duration output')
print(network_output[1][0])

pitch input
[460 460 460 460 460 460 460 460 460 460 460 460 460 460 460 460 460 460
 460 460 460 460 460 460 460 460 460 460 460 460 460 460]
duration input
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
pitch output
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.